# Лабораторная работа №5
## Вариант 3
### Выполнил: Федоров А. В. 6211

### Импорты библиотек:

In [5]:
import torch
from torch import nn
from torch.optim import Adam
from torchvision.transforms import Compose
from torchvision.transforms import ToTensor
from torchvision.transforms import Resize
from torchvision.transforms import Normalize
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import os
from typing import Tuple, Any
from pandas import read_csv
from random import randint
from random import shuffle
import cv2
import matplotlib.pyplot as plt
from numpy import array


### Задание 1. Загружаю исходный датасет:

In [6]:
data_frame = read_csv('annotation_original.csv')
df = data_frame[["Absolute path", "Class label"]]
df.loc[df['Class label'] == 'cat', 'Class label'] = 0
df.loc[df['Class label'] == 'dog', 'Class label'] = 1
df.to_csv("annotation.csv", index=0)
print(df)

                                          Absolute path Class label
0     C:\Users\Professional\PycharmProjects\lab3\dat...           0
1     C:\Users\Professional\PycharmProjects\lab3\dat...           0
2     C:\Users\Professional\PycharmProjects\lab3\dat...           0
3     C:\Users\Professional\PycharmProjects\lab3\dat...           0
4     C:\Users\Professional\PycharmProjects\lab3\dat...           0
...                                                 ...         ...
2095  C:\Users\Professional\PycharmProjects\lab3\dat...           1
2096  C:\Users\Professional\PycharmProjects\lab3\dat...           1
2097  C:\Users\Professional\PycharmProjects\lab3\dat...           1
2098  C:\Users\Professional\PycharmProjects\lab3\dat...           1
2099  C:\Users\Professional\PycharmProjects\lab3\dat...           1

[2100 rows x 2 columns]


C:\Users\Professional\AppData\Local\Temp\ipykernel_20308\1360825724.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[df['Class label'] == 'cat', 'Class label'] = 0
C:\Users\Professional\AppData\Local\Temp\ipykernel_20308\1360825724.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[df['Class label'] == 'dog', 'Class label'] = 1


### Класс датасета из лекции:

In [7]:
class CustomImageDataset(Dataset):
    def __init__(self, path_to_annotation_file: str, transform: Any = None, target_transform: Any = None) -> None:
        self.path_to_annotation_file = path_to_annotation_file
        self.dataset_info = read_csv(path_to_annotation_file, header=None)
        self.dataset_info.drop(self.dataset_info.columns[[0]], axis=1, inplace=True)
        self.dataset_info.drop(index=0, axis=0, inplace=True)
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self) -> int:
        return len(self.dataset_info)

    def __getitem__(self, index: int) -> Tuple[torch.tensor, int]:
        path_to_image = self.dataset_info.iloc[index, 0]
        image = cv2.cvtColor(cv2.imread(path_to_image), cv2.COLOR_BGR2RGB)
        label = self.dataset_info.iloc[index, 1]

        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_Transform(label)

        return image, label

### Задание 2.

In [8]:
random_list = list(range(2100))
shuffle(random_list)
train_csv = df.iloc[random_list[:1676]]
train_csv.to_csv("train_csv.csv")
test_csv = df.iloc[random_list[1676:1888]]
test_csv.to_csv("test_csv.csv")
valid_csv = df.iloc[random_list[1888:]]
valid_csv.to_csv("valid_csv.csv")
print(len(train_csv), len(test_csv), len(valid_csv))

1676 212 212
